In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()

In [2]:
sc = spark.sparkContext  # 필요 시 RDD API를 위해 사용

In [8]:
sc.defaultParallelism

2

In [9]:
sc.getConf().getAll()

[('spark.app.startTime', '1754273544353'),
 ('spark.app.name', 'restaurant-review-average'),
 ('spark.app.submitTime', '1754273544205'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '34063'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNN

In [10]:
data = [
    (0, "짜장면", "중식", 125),
    (1, "짬뽕", "중식", 235),
    (2, "김밥", "분식", 32),
    (3, "떡볶이", "분식", 534),
    (4, "라멘", "일식", 223),
    (5, "돈가스", "일식", 52),
    (6, "우동", "일식", 12),
    (7, "쌀국수", "아시안", 312),
    (8, "햄버거", "패스트푸드", 12),
    (9, "치킨", "패스트푸드", 23),
]

In [12]:
rdd1 = sc.parallelize(data)

In [32]:
lines = sc.textFile("file:///home/jovyan/work/learning_spark_data/restaurant_reviews.csv")

lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [33]:
# 총 건수 확인

# 첫줄, 데이터 분할
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)
# map으로 split
#lines.map(lambda x : x.split(','))

In [35]:
filtered_lines.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [36]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2] #카테고리
    
    # reviews는 정수로 parse
    reviews = fields[3] # 리뷰수
    reviews = int(reviews)
    
    return category, reviews

In [15]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [37]:
category_reviews = filtered_lines.map(parse)
category_reviews

PythonRDD[40] at RDD at PythonRDD.scala:53

In [19]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [21]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
# map 이랑 다르게 mapvalues는 values만 바뀜
# 뒤에 추가한 숫자는 메뉴 수
category_review_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [22]:
#같은 key(카테고리)끼리 x와 y의 (리뷰 수 합계, 개수 합계)를 누적
#첫번째 중식 x, 두번째 중식 y , [0] 은 리뷰수, [1] 은 메뉴 개수 -> 같은 키끼리 행을 바꿔가며 계속 누적해 나간다.
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [23]:
average = reduced.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

# persist를 사용하는 경우

In [38]:
#job x
categoryReviews = filtered_lines.map(parse).persist() # categoryReviews RDD는 하나만 존재하는 RDD
categoryReviews

PythonRDD[41] at RDD at PythonRDD.scala:53

In [39]:
result3 = categoryReviews.take(10)
result3

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [26]:
result4 = categoryReviews.mapValues(lambda x : (x, 1)).collect()
result4

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [40]:
# 카테고리별 값1 합계 계산 (첫 번째 연산)
result_sum = categoryReviews \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
result_sum

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

In [41]:
# (카테고리, (점수, 1)) 형태로 변환
rdd_with_count = categoryReviews.map(lambda x: (x[0], (x[1], 1)))
rdd_with_count.take(3)

[('중식', (125, 1)), ('중식', (235, 1)), ('분식', (32, 1))]

In [42]:
# reduceByKey로 (점수 총합, 개수 총합) 집계
rdd_sums = rdd_with_count.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
rdd_sums.take(3)

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [43]:
# 평균 계산
rdd_avg = rdd_sums.mapValues(lambda x: round(x[0] / x[1], 2))

# 결과 출력
rdd_avg.collect()


[('중식', 180.0), ('분식', 283.0), ('일식', 95.67), ('아시안', 312.0), ('패스트푸드', 17.5)]

In [44]:
# 기존 저장 레벨 해제
categoryReviews.unpersist()

PythonRDD[41] at RDD at PythonRDD.scala:53

In [45]:
from pyspark import StorageLevel
categoryReviews.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[41] at RDD at PythonRDD.scala:53

In [46]:
# 구조 확인하기
categoryReviews.take(3)

[('중식', 125), ('중식', 235), ('분식', 32)]

In [47]:
result11 = categoryReviews.filter(lambda x: x[1] > 100).count()
result12 = categoryReviews.filter(lambda x: x[1] <= 100).count()

# Narrow Transformations
1:1 변환 -> 하나의 열을 다룰 떄 다른 데이터가 필요없는 경우

filter(), map(), flatMap(), sample(), union()

# flatMap()


In [48]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

[[1, 2], [2, 3], [3, 4]]

In [49]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

[1, 2, 2, 3, 3, 4]

In [ ]:
# 텍스트를 단어로 쪼개서 flatMap()

In [50]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [51]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[63] at readRDDFromFile at PythonRDD.scala:289

In [52]:
#job 추가 x
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [ ]:
# 집합 Transformation

In [53]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [54]:
num1.intersection(num2).collect()

[4, 5]

In [ ]:
# 합집합 구하기 - union

In [55]:
num_union = num1.union(num2)
num_union

UnionRDD[77] at union at NativeMethodAccessorImpl.java:0

In [56]:
num_union.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
# 차집합 구하기 - subtract

In [57]:
num1.subtract(num2).collect()

[1, 2, 3]

In [ ]:
# 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)


In [58]:
# withReplacement : True -> 중복 추출
num_union.sample(True, 0.3).collect()

[2, 5]

In [59]:
# withReplacement : False -> 중복 X
num_union.sample(False, 0.7).collect()

[1, 3, 4, 4, 5, 6, 8, 9]

In [60]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
num_union.sample(True, 0.5, seed=42).collect()

[4, 5, 5, 5, 7]

# Wide Transformations

In [61]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[89] at readRDDFromFile at PythonRDD.scala:289

In [62]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[94] at RDD at PythonRDD.scala:53

In [63]:
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f4650131590>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f4650131350>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f4650131610>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f4650c39710>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f4650131750>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f4650c417d0>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f4650c39690>)]

In [ ]:
# 내로우 narrow  트랜스포메이션

In [64]:
### 2. Narrow 트랜스포메이션 예제
numbers = sc.parallelize([1, 2, 3, 4, 5])

# 1:1 변환 (Narrow Transformation)
sample_rdd = numbers.sample(False, 0.5)
sample_rdd.take(3)

[1, 3]

In [65]:
union_rdd = numbers.union(sc.parallelize([6,7,8]))
union_rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8]

In [67]:
#wide 연산
kv_rdd = sc.parallelize([('apple', 1),('banana',2),('cherry',3)])

In [68]:
mapp_values = kv_rdd.mapValues(lambda x:x*10)
mapp_values.collect()

[('apple', 10), ('banana', 20), ('cherry', 30)]

In [69]:
grouped_rdd = kv_rdd.groupByKey()
grouped_rdd.collect()

[('apple', <pyspark.resultiterable.ResultIterable at 0x7f4633b90d90>),
 ('banana', <pyspark.resultiterable.ResultIterable at 0x7f4650c408d0>),
 ('cherry', <pyspark.resultiterable.ResultIterable at 0x7f463395be50>)]

In [71]:
reduced_rdd = kv_rdd.reduceByKey(lambda x,y : x+y)
reduced_rdd.collect()

[('apple', 1), ('banana', 2), ('cherry', 3)]

In [73]:
# 두 RDD 조인하기
rdd1 = sc.parallelize([("apple", 2), ("banana", 1)])
rdd2 = sc.parallelize([("apple", "fruit"), ("banana", "fruit"), ("carrot", "vegetable")])

# 두 RDD 조인
joined_rdd = rdd1.join(rdd2)
print(joined_rdd.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [75]:
distinct_rdd = sc.parallelize([1, 1, 2, 2, 3, 3]).distinct()

In [78]:
print("Distinct RDD:", distinct_rdd.collect())  # [1, 2, 3]
print("Joined RDD:", joined_rdd.collect())  # [("apple", (2, "fruit")), ("banana", (1, "fruit"))]

Distinct RDD: [2, 1, 3]
Joined RDD: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [79]:
# 왼쪽 외부 조인 (leftOuterJoin)
left_joined = rdd1.leftOuterJoin(rdd2)
print(left_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [80]:
# 오른쪽 외부 조인 (rightOuterJoin)
right_joined = rdd1.rightOuterJoin(rdd2)
print(right_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]


In [81]:
# subtractByKey 연산
subtract_result = rdd1.subtractByKey(sc.parallelize([("apple", "any")]))
print(subtract_result.collect())
# 출력 결과: [('banana', 1)]

[('banana', 1)]


In [83]:
#spark.stop()